# Скрипт для формування pop-up 


#### Ініціалізація скрипту

У змінну `current_folder` вкажіть директорію де будуть зберігатися файли

In [ ]:
# -*- coding: utf-8 -*-

import arcpy
import csv
import json

aprx = arcpy.mp.ArcGISProject("current")

current_folder = "C:\\Users\\Home PC\\Desktop"

Список усіх доступних карт в поточному проекті

In [ ]:
for numb, map in enumerate(aprx.listMaps()):
    print(numb, map.name)

У змінну `map_number` ввести номер відповідної карти

In [ ]:
map_number = 0
map = aprx.listMaps()[map_number]
layers = map.listLayers()

#### Створення вихідних даних

In [ ]:
setFilelds = set()

for layer in layers:
    if layer.isFeatureLayer:
        path = arcpy.Describe(layer).catalogPath
        fields = arcpy.ListFields(path)
        for f in fields:
            setFilelds.add(f.name)

csv_header = ["fieldName", "label", "visible"]
csv_data = [[row, "", ""] for row in setFilelds]

with open(f"{current_folder}\\pop_up.csv", 'w', newline='', encoding="utf8") as f:
    writer = csv.writer(f)
    writer.writerow(csv_header)
    writer.writerows(csv_data)

<div class="alert alert-block alert-info">

⚠️ **Важливо!** У робочій директорії зберігся файл `pop_up.csv`  Цей файл потрібно заповнити перед запуском наступних команд. Можливо будуть проблеми з кодуванням, рекомендую csv відкривати у google spreadsheet
</div>



| fieldName | label | visible |
|-----------|-------|---------|
| street    |       |         |
| OID       |       |         |
| Напр      |       |         |
| .....     |       |         |

- Колонка fieldName заповняється автоматично, це всі унікальні значення назв параметрів
- label - це назва поля яка буде відображатися у pop-up
- visible - це значення `True/False` відповідають чи буде показаний параметр у pop-up чи ні
  

| fieldName | label        | visible |
|-----------|--------------|---------|
| street    | Назва вулиці | True    |
| OID       | OID          | False   |
| Напр      | Напруга, кВ  | True    |
| .....     | .....        | ....    |

In [58]:
csv_file = open(f"{current_folder}\\pop_up.csv", 'r', encoding="utf8")

reader = csv.reader(csv_file)
header = next(reader)

def parse(string):
    d = {'TRUE': True, 'FALSE': False, 'True': True, 'False': False}
    return d.get(string, False)

csv_dict = {row[0]:{"label":row[1], "visible":parse(row[2])} for row in reader}

#### Формування JSON

In [59]:
list_layers = []

for n, layer in enumerate(layers):
    if layer.isGroupLayer:
        list_layers.append(
            {
                "id": n
            }
        )
    elif layer.isFeatureLayer:
        path = arcpy.Describe(layer).catalogPath
        fields = arcpy.ListFields(path)
        
        fieldsList = []
        
        for field in fields:
            if field.type == "Double":
                fieldsList.append(
                    {
                        "fieldName": field.name,
                        "label": csv_dict[field.name]['label'],
                        "tooltip": "",
                        "visible": csv_dict[field.name]['visible'],
                        "stringFieldOption": "textbox",
                        "format": {
                            "places": 2,
                            "digitSeparator": True
                        }
                    }
                )
            else:
                fieldsList.append(
                    {
                        "fieldName": field.name,
                        "label": csv_dict[field.name]['label'],
                        "tooltip": "",
                        "visible": csv_dict[field.name]['visible'],
                        "stringFieldOption": "textbox"
                    }
                )
            
        
        list_layers.append(
            {
               "id": n,
                "popupInfo": {
                    "title": layer.name,
                    "fieldInfos": fieldsList,
                    "description": None,
                    "showAttachments": True,
                    "mediaInfos": []
                } 
            }
        )

jsonList = json.dumps(list_layers, ensure_ascii=False)

with open(f"{current_folder}\\pop_up.json", 'w', encoding="utf8") as outfile:
    outfile.write(jsonList)  

У робочій директорії зберігся готовий файл `pop_up.json`

У ArcGIS Assistant потрібно знайти відповідий шар і у значення `layer: []` замінити даними із json разом із квадратними скобками 